# Display some images

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import glob

In [ ]:
paths = glob.glob('/root/data/sflab_ground_truth/v2_071218/raw_images/*')

In [ ]:
random_path = np.random.choice(paths)
Image.open(random_path).resize((400, 300))

In [ ]:
img = cv2.imread(random_path)
dst = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)

In [ ]:
# plt.figure(figsize=(15, 15))
plt.imshow(cv2.resize(dst, (400, 300)))

# Some data preprocessing

In [ ]:
import json
import os
import numpy
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import csv

In [ ]:
labels = json.load(open('/root/data/sflab_ground_truth/v2_071218/sflab_v2_labels_0.json'))

In [ ]:
# just make the data easier
for label in labels:
    local = '/root/data/sflab_ground_truth/v2_071218/raw_images/' + label['External ID']
    label['path'] = local

### Create masks && bouding boxes

In [ ]:
# get the data with masks
annotated_data = []
for l in labels:
    if 'Masks' in l:
        annotated_data.append(l)
print(len(annotated_data))

In [ ]:
# create masks and save them
bboxes = []
for data in annotated_data:
    img = Image.new('L', (4096, 3000), 0)
    for fish in data['Label']['fish']:
        polygon = [(int(k['x']), 3000 - int(k['y'])) for k in fish]
        x1 = min([g[0] for g in polygon])
        x2 = max([g[0] for g in polygon])
        y1 = min([g[1] for g in polygon])
        y2 = max([g[1] for g in polygon])
        bboxes.append([data['path'], x1, y1, x2, y2, 'fish'])
        ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = numpy.array(img)
    mask_path = '/root/data/sflab_ground_truth/v2_071218/semantic_masks/' + data['External ID'].split('.')[0] + '.npy'
    data['mask_path'] = mask_path
    # np.save(mask_path, mask)

In [ ]:
# display some masks
masks = glob.glob('/root/data/sflab_ground_truth/v2_071218/semantic_masks/*')
plt.imshow(np.load(np.random.choice(masks)))

In [ ]:
bbox = bboxes[2]

In [ ]:
# display some bounding boxes
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(np.array(Image.open(bbox[0])))

# Create a Rectangle patch
rect = patches.Rectangle((bbox[1],bbox[2]),bbox[3]-bbox[1], bbox[4]-bbox[2],
                         linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

In [ ]:
with open('/root/data/sflab_ground_truth/v2_071218/fish.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for bbox in bboxes:
        writer.writerow(bbox)

### Truncate images and masks because of reflection

In [ ]:
trunc_bboxes = []
for data in annotated_data:
    
    # load the mask
    mask = np.load(data['mask_path'])
    mask_name = os.path.basename(data['mask_path'])
    img_name = os.path.basename(data['path'])
    # get the first non zero coordinate - suboptimal
    zero_coord = np.where(np.sum(mask, axis=1) == 0)
    highest_y = 0
    for zc in zero_coord[0]:
        if zc == highest_y:
            highest_y += 1
        else:
            break
    # truncate the mask and the images
    if highest_y > 50:
        trunc_mask = mask[highest_y-50:, :]
        img = np.array(Image.open(data['path']))[highest_y-50:, :]
    else:
        trunc_mask = mask[highest_y:, :]
        img = np.array(Image.open(data['path']))[highest_y:, :]
    np.save('/root/data/sflab_ground_truth/v2_071218/trunc_semantic_masks/' + mask_name, trunc_mask)
    
    
    Image.fromarray(img).save('/root/data/sflab_ground_truth/v2_071218/trunc_images/' + img_name)
    
    # truncate the bounding boxes
    ct = 0
    for bb in bboxes:
        if ct > 3:
            break
        if bb[0] == data['path']:
            ct +1
            if highest_y > 50:
                trunc_bboxes.append(['/root/data/sflab_ground_truth/v2_071218/trunc_images/' + img_name, bb[1], bb[2]-highest_y+50, bb[3], bb[4]-highest_y+50,bb[5]])
            else:
                trunc_bboxes.append(['/root/data/sflab_ground_truth/v2_071218/trunc_images/' + img_name, bb[1], bb[2]-highest_y, bb[3], bb[4]-highest_y,bb[5]])                


In [ ]:
with open('/root/data/sflab_ground_truth/v2_071218/trunc_fish.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for bbox in trunc_bboxes:
        writer.writerow(bbox)

In [ ]:
import glob

In [ ]:
# display some truncated masks
masks = glob.glob('/root/data/sflab_ground_truth/v2_071218/trunc_semantic_masks/*')

In [ ]:
random_mask = np.random.choice(masks)
mask_id = os.path.basename(random_mask).split('.')[0]
random_image = '/root/data/sflab_ground_truth/v2_071218/trunc_images/{}.jpg'.format(mask_id)
plt.imshow(np.array(Image.open(random_image)))
plt.imshow(np.load(random_mask), alpha=0.1)


In [ ]:
bbox = [b for b in trunc_bboxes if b[0] == '/root/data/sflab_ground_truth/v2_071218/trunc_images/right_1531429803055.jpg'][0]

In [ ]:
# display some bounding boxes
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(np.array(Image.open(bbox[0])))

# Create a Rectangle patch
rect = patches.Rectangle((bbox[1],bbox[2]),bbox[3]-bbox[1], bbox[4]-bbox[2],
                         linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

### bboxes debug

In [ ]:
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image

In [ ]:
dataset = []
with open('/root/data/sflab_ground_truth/v2_071218/trunc_fish.csv', 'r') as f:
    reader = csv.reader(f)
    for l in reader:
        dataset.append(l)

In [ ]:
bbox = dataset[np.random.randint(0, high=len(dataset))]

In [ ]:
bbox = ['/root/data/sflab_ground_truth/v2_071218/trunc_images/right_1531434392670.jpg','2406','100','4107','1002','fish']

In [ ]:
# display some bounding boxes
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(np.array(Image.open(bbox[0])))

# Create a Rectangle patch
coord = [0] + [int(b) for b in bbox[1:-1]]
rect = patches.Rectangle((coord[1],coord[2]),coord[3]-coord[1], coord[4]-coord[2],
                         linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

In [ ]:
fixed_dataset = []
for data in dataset:
    bbox = [np.max([int(b), 1]) for b in data[1:-1]]
    
    if bbox[0] > 4095:
        bbox[0] = 4095
    if bbox[1] > 2999:
        bbox[1] = 2999
    if bbox[2] > 4095:
        bbox[2] = 4095
    if bbox[3] > 2999:
        bbox[3] = 2999
    fixed_dataset.append([data[0]] + bbox + ['fish'])

In [ ]:
fixed_dataset

In [ ]:
with open('/root/data/sflab_ground_truth/v2_071218/trunc_fish2.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for bbox in fixed_dataset:
        writer.writerow(bbox)

### find the bugs

In [ ]:
dataset = []
with open('/root/data/sflab_ground_truth/v2_071218/trunc_fish2.csv', 'r') as f:
    reader = csv.reader(f)
    for l in reader:
        dataset.append(l)

In [ ]:
for annotations in dataset:
    image = np.array(Image.open(annotations[0]))
    
    annotations[1] = int(annotations[1])
    annotations[2] = int(annotations[2])
    annotations[3] = int(annotations[3])
    annotations[4] = int(annotations[4])
    invalid_indices = np.where(
        (annotations[3] <= annotations[1]) |
        (annotations[4] <= annotations[2]) |
        (annotations[1] < 0) |
        (annotations[2] < 0) |
        (annotations[3] > image.shape[1]) |
        (annotations[4] > image.shape[0])
    )[0]
    if len(invalid_indices):
        print(image.shape)
        print(annotations)
        print('issue')